# Робота з людською участю за допомогою Microsoft Agent Framework

## 🎯 Цілі навчання

У цьому ноутбуці ви навчитеся впроваджувати робочі процеси **з людською участю** за допомогою `RequestInfoExecutor` у Microsoft Agent Framework. Цей потужний підхід дозволяє призупиняти роботу AI для отримання вводу від людини, роблячи ваших агентів інтерактивними та надаючи людям контроль над важливими рішеннями.

## 🔄 Що таке робота з людською участю?

**Робота з людською участю (HITL)** — це дизайн-патерн, у якому AI-агенти призупиняють виконання, щоб запросити ввід від людини перед продовженням. Це важливо для:

- ✅ **Критичних рішень** - Отримання схвалення від людини перед виконанням важливих дій
- ✅ **Неоднозначних ситуацій** - Дозволити людям уточнити, коли AI не впевнений
- ✅ **Переваг користувача** - Запитати користувача про вибір між кількома варіантами
- ✅ **Відповідності та безпеки** - Забезпечити контроль людини для регульованих операцій
- ✅ **Інтерактивного досвіду** - Створення розмовних агентів, які реагують на ввід користувача

## 🏗️ Як це працює в Microsoft Agent Framework

Фреймворк надає три ключові компоненти для HITL:

1. **`RequestInfoExecutor`** - Спеціальний виконавець, який призупиняє робочий процес і генерує `RequestInfoEvent`
2. **`RequestInfoMessage`** - Базовий клас для типізованих запитів, які надсилаються людям
3. **`RequestResponse`** - Зв'язує відповіді людей з оригінальними запитами за допомогою `request_id`

**Шаблон робочого процесу:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Наш приклад: Бронювання готелю з підтвердженням користувача

Ми розширимо умовний робочий процес, додавши підтвердження від людини **перед** пропозицією альтернативних напрямків:

1. Користувач запитує напрямок (наприклад, "Париж")
2. `availability_agent` перевіряє наявність номерів
3. **Якщо номерів немає** → `confirmation_agent` запитує: "Чи хочете ви побачити альтернативи?"
4. Робочий процес **призупиняється** за допомогою `RequestInfoExecutor`
5. **Людина відповідає** "так" або "ні" через консольний ввід
6. `decision_manager` спрямовує залежно від відповіді:
   - **Так** → Показати альтернативні напрямки
   - **Ні** → Скасувати запит на бронювання
7. Відображення остаточного результату

Це демонструє, як надати користувачам контроль над пропозиціями агента!

---

Почнемо! 🚀


## Крок 1: Імпорт необхідних бібліотек

Ми імпортуємо стандартні компоненти Agent Framework, а також **класи, специфічні для взаємодії з людиною**:
- `RequestInfoExecutor` - Виконавець, який призупиняє робочий процес для отримання вводу від людини
- `RequestInfoEvent` - Подія, яка виникає при запиті вводу від людини
- `RequestInfoMessage` - Базовий клас для типізованих даних запиту
- `RequestResponse` - Зв'язує відповіді людини із запитами
- `WorkflowOutputEvent` - Подія для виявлення результатів робочого процесу


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Крок 2: Визначення моделей Pydantic для структурованих результатів

Ці моделі визначають **схему**, яку агенти будуть повертати. Ми зберігаємо всі моделі з умовного робочого процесу та додаємо:

**Нові для участі людини:**
- `HumanFeedbackRequest` - Підклас `RequestInfoMessage`, який визначає дані запиту, що надсилаються людям
  - Містить `prompt` (питання для запиту) та `destination` (контекст про недоступне місто)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Крок 3: Створення інструменту бронювання готелів

Той самий інструмент з умовного робочого процесу - перевіряє, чи є доступні номери в пункті призначення.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Крок 4: Визначення функцій умов для маршрутизації

Нам потрібні **чотири функції умов** для нашого процесу з участю людини:

**З умовного робочого процесу:**
1. `has_availability_condition` - Маршрутизує, коли готелі ДОСТУПНІ
2. `no_availability_condition` - Маршрутизує, коли готелі НЕ ДОСТУПНІ

**Нові для процесу з участю людини:**
3. `user_wants_alternatives_condition` - Маршрутизує, коли користувач відповідає "так" на альтернативи
4. `user_declines_alternatives_condition` - Маршрутизує, коли користувач відповідає "ні" на альтернативи


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Крок 5: Створення Виконавця Менеджера Рішень

Це **основа патерну "людина в циклі"!** `DecisionManager` — це спеціальний `Executor`, який:

1. **Отримує зворотний зв'язок від людини** через об'єкти `RequestResponse`
2. **Обробляє рішення користувача** (так/ні)
3. **Спрямовує робочий процес**, надсилаючи повідомлення відповідним агентам

Основні особливості:
- Використовує декоратор `@handler`, щоб зробити методи доступними як кроки робочого процесу
- Отримує `RequestResponse[HumanFeedbackRequest, str]`, що містить як початковий запит, так і відповідь користувача
- Генерує прості повідомлення "так" або "ні", які активують наші функції умов


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Крок 6: Створення власного виконавця відображення

Той самий виконавець відображення з умовного робочого процесу - видає остаточні результати як вихід робочого процесу.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Крок 7: Завантаження змінних середовища

Налаштуйте клієнт LLM (GitHub Models, Azure OpenAI або OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Крок 8: Створення AI агентів та виконавців

Ми створюємо **шість компонентів робочого процесу**:

**Агенти (загорнуті в AgentExecutor):**
1. **availability_agent** - Перевіряє доступність готелів за допомогою інструменту
2. **confirmation_agent** - 🆕 Готує запит на підтвердження від людини
3. **alternative_agent** - Пропонує альтернативні міста (коли користувач погоджується)
4. **booking_agent** - Заохочує до бронювання (коли є доступні номери)
5. **cancellation_agent** - 🆕 Обробляє повідомлення про скасування (коли користувач відмовляється)

**Спеціальні виконавці:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, який призупиняє робочий процес для введення даних людиною
7. **decision_manager** - 🆕 Спеціальний виконавець, який маршрутизує на основі відповіді людини (вже визначено вище)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Крок 9: Створення робочого процесу з участю людини

Тепер ми створюємо граф робочого процесу з **умовним маршрутом**, включаючи шлях з участю людини:

**Структура робочого процесу:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Ключові зв'язки:**
- `availability_agent → confirmation_agent` (коли немає кімнат)
- `confirmation_agent → prepare_human_request` (зміна типу)
- `prepare_human_request → request_info_executor` (пауза для участі людини)
- `request_info_executor → decision_manager` (завжди - надає RequestResponse)
- `decision_manager → alternative_agent` (коли користувач каже "так")
- `decision_manager → cancellation_agent` (коли користувач каже "ні")
- `availability_agent → booking_agent` (коли кімнати доступні)
- Усі шляхи закінчуються на `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Крок 10: Запустіть тестовий випадок 1 - Місто БЕЗ доступності (Париж із підтвердженням людиною)

Цей тест демонструє **повний цикл із залученням людини**:

1. Запит на готель у Парижі
2. availability_agent перевіряє → Немає номерів
3. confirmation_agent створює питання для людини
4. request_info_executor **призупиняє робочий процес** і видає `RequestInfoEvent`
5. **Додаток виявляє подію та запитує користувача через консоль**
6. Користувач вводить "так" або "ні"
7. Додаток надсилає відповідь через `send_responses_streaming()`
8. decision_manager маршрутизує залежно від відповіді
9. Відображається остаточний результат

**Основний шаблон:**
- Використовуйте `workflow.run_stream()` для першої ітерації
- Використовуйте `workflow.send_responses_streaming(pending_responses)` для наступних ітерацій
- Слухайте `RequestInfoEvent`, щоб визначити, коли потрібен ввід від людини
- Слухайте `WorkflowOutputEvent`, щоб отримати остаточні результати


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Крок 11: Запустіть тестовий випадок 2 - Місто З Наявністю (Стокгольм - Без необхідності втручання людини)

Цей тест демонструє **прямий шлях**, коли кімнати доступні:

1. Запит на готель у Стокгольмі
2. availability_agent перевіряє → Кімнати доступні ✅
3. booking_agent пропонує бронювання
4. display_result показує підтвердження
5. **Втручання людини не потрібне!**

Робочий процес повністю обходить шлях із залученням людини, коли кімнати доступні.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Основні висновки та найкращі практики для "людина в циклі"

### ✅ Що ви дізналися:

#### 1. **Шаблон RequestInfoExecutor**
Шаблон "людина в циклі" у Microsoft Agent Framework використовує три ключові компоненти:
- `RequestInfoExecutor` - Зупиняє робочий процес і генерує події
- `RequestInfoMessage` - Базовий клас для типізованих запитів (створюйте підкласи!)
- `RequestResponse` - Зв'язує відповіді людини з початковими запитами

**Критичне розуміння:**
- `RequestInfoExecutor` НЕ збирає введення самостійно - він лише зупиняє робочий процес
- Ваш код програми повинен слухати `RequestInfoEvent` і збирати введення
- Ви повинні викликати `send_responses_streaming()` з dict, що зв'язує `request_id` з відповіддю користувача

#### 2. **Шаблон потокового виконання**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Архітектура, керована подіями**
Слухайте конкретні події для управління робочим процесом:
- `RequestInfoEvent` - Потрібне введення людини (робочий процес зупинено)
- `WorkflowOutputEvent` - Доступний кінцевий результат (робочий процес завершено)
- `WorkflowStatusEvent` - Зміни стану (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS тощо)

#### 4. **Користувацькі виконавці з @handler**
`DecisionManager` демонструє, як створювати виконавців, які:
- Використовують декоратор `@handler` для відкриття методів як кроків робочого процесу
- Отримують типізовані повідомлення (наприклад, `RequestResponse[HumanFeedbackRequest, str]`)
- Направляють робочий процес, надсилаючи повідомлення іншим виконавцям
- Доступ до контексту через `WorkflowContext`

#### 5. **Умовна маршрутизація з рішеннями людини**
Ви можете створювати функції умов, які оцінюють відповіді людини:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Реальні застосування:

1. **Процеси затвердження**
   - Отримання схвалення менеджера перед обробкою звітів про витрати
   - Необхідність людського перегляду перед відправкою автоматизованих електронних листів
   - Підтвердження транзакцій з високою вартістю перед виконанням

2. **Модерація контенту**
   - Позначення сумнівного контенту для перегляду людиною
   - Запит модераторів щодо прийняття остаточного рішення у складних випадках
   - Ескалація до людини, коли впевненість AI низька

3. **Обслуговування клієнтів**
   - Дозволити AI автоматично обробляти рутинні запитання
   - Ескалація складних питань до людських агентів
   - Запит клієнта, чи хоче він поговорити з людиною

4. **Обробка даних**
   - Запит людей для вирішення неоднозначних записів даних
   - Підтвердження AI-інтерпретацій неясних документів
   - Дозволити користувачам вибирати між кількома допустимими інтерпретаціями

5. **Системи критичної безпеки**
   - Вимагати підтвердження людини перед незворотними діями
   - Отримання схвалення перед доступом до конфіденційних даних
   - Підтвердження рішень у регульованих галузях (охорона здоров'я, фінанси)

6. **Інтерактивні агенти**
   - Створення розмовних ботів, які задають додаткові запитання
   - Створення майстрів, які проводять користувачів через складні процеси
   - Дизайн агентів, які співпрацюють з людьми крок за кроком

### 🔄 Порівняння: З "людина в циклі" та без

| Функція | Умовний робочий процес | Робочий процес "людина в циклі" |
|---------|---------------------|---------------------------|
| **Виконання** | Один `workflow.run()` | Цикл з `run_stream()` + `send_responses_streaming()` |
| **Введення користувача** | Відсутнє (повністю автоматизовано) | Інтерактивні запити через `input()` або UI |
| **Компоненти** | Агенти + Виконавці | + RequestInfoExecutor + DecisionManager |
| **Події** | Лише AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent тощо |
| **Зупинка** | Без зупинки | Робочий процес зупиняється на RequestInfoExecutor |
| **Контроль людини** | Без контролю людини | Люди приймають ключові рішення |
| **Випадок використання** | Автоматизація | Співпраця та контроль |

### 🚀 Розширені шаблони:

#### Кілька точок прийняття рішень людиною
Ви можете мати кілька вузлів `RequestInfoExecutor` в одному робочому процесі:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Обробка тайм-аутів
Реалізуйте тайм-аут для відповідей людини:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Інтеграція з багатим UI
Замість `input()` інтегруйте з веб-інтерфейсом, Slack, Teams тощо:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Умовний "людина в циклі"
Запитуйте введення людини лише в конкретних ситуаціях:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Найкращі практики:

1. **Завжди створюйте підкласи RequestInfoMessage**
   - Забезпечує типову безпеку та перевірку
   - Дозволяє багатий контекст для рендерингу UI
   - Уточнює намір кожного типу запиту

2. **Використовуйте описові запити**
   - Включайте контекст того, що ви запитуєте
   - Пояснюйте наслідки кожного вибору
   - Зберігайте запитання простими та зрозумілими

3. **Обробляйте несподіване введення**
   - Перевіряйте відповіді користувачів
   - Надавайте значення за замовчуванням для недійсного введення
   - Надавайте чіткі повідомлення про помилки

4. **Відстежуйте ID запитів**
   - Використовуйте зв'язок між request_id і відповідями
   - Не намагайтеся керувати станом вручну

5. **Проектуйте для неблокування**
   - Не блокуйте потоки, очікуючи введення
   - Використовуйте асинхронні шаблони
   - Підтримуйте одночасні екземпляри робочого процесу

### 📚 Суміжні концепції:

- **Посередник агента** - Перехоплення викликів агента (попередній блокнот)
- **Управління станом робочого процесу** - Збереження стану робочого процесу між запусками
- **Співпраця між агентами** - Поєднання "людина в циклі" з командами агентів
- **Архітектури, керовані подіями** - Створення реактивних систем з подіями

---

### 🎓 Вітаємо!

Ви освоїли робочі процеси "людина в циклі" з Microsoft Agent Framework! Тепер ви знаєте, як:
- ✅ Зупиняти робочі процеси для збору введення людини
- ✅ Використовувати RequestInfoExecutor і RequestInfoMessage
- ✅ Обробляти потокове виконання за допомогою подій
- ✅ Створювати користувацькі виконавці з @handler
- ✅ Маршрутизувати робочі процеси на основі рішень людини
- ✅ Створювати інтерактивних AI-агентів, які співпрацюють з людьми

**Це критично важливий шаблон для створення надійних, контрольованих AI-систем!** 🚀



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
